# Importing libraries 

In [1]:
import numpy as np
import cv2


# Extraction des contours 

In [5]:
class ExtractEdges:
    def __init__(self,):
        pass
    def filtre(self,img, kernel):
        newimg = np.zeros(img.shape, dtype=np.float32)

        kernel_height, kernel_width = kernel.shape
        kh, kw = kernel_height // 2, kernel_width // 2  

        for i in range(kh, img.shape[0] - kh):
            for j in range(kw, img.shape[1] - kw):
                sum_value = 0
                for ki in range(kernel_height):
                    for kj in range(kernel_width):
                        sum_value += kernel[ki, kj] * img[i + ki - kh, j + kj - kw]

                newimg[i, j] = sum_value
        return newimg
    def sobel(self, img):
        # gradient selon x 
        kernel = np.array([[-1, 0, 1], [-2,0,2], [-1, 0, 1]])/5
        gradx = self.filtre(img, kernel)
        # gradient selon y 
        kernel = np.array([[1, 2, 1], [0,0,0], [-1, -2, -1]])/5
        grady = self.filtre(img, kernel)    
        # image final 
        newimg = np.sqrt(gradx**2 + grady**2)
        newImg = cv2.threshold(newimg, 128, 1,cv2.THRESH_BINARY )
        return newImg[1]
    def prewitt(self, img):
        # gradient selon x 
        kernel = np.array([[-1, 0, 1], [-1,0,1], [-1, 0, 1]])/3
        gradx = self.filtre(img, kernel)


        # gradient selon y 
        kernel = np.array([[-1, -1, -1], [0,0,0], [1, 1, 1]])/3
        grady = self.filtre(img, kernel)

        # image final 
        newimg = np.sqrt(gradx**2 + grady**2)
        newImg = cv2.threshold(newimg, 90, 1,cv2.THRESH_BINARY)
        return newImg[1]
    def robinson(self, img, level):
        if level == 3:
            c = 1
            n = 3
        elif level == 5:
            c = 2
            n = 5
        else:
            return "the level should be 3 or 5"
        # gradient niveau 1 
        kernel = np.array([[1,c,1], [0, 0, 0], [-1, -c, -1]])/n
        grad1 = self.filtre(img, kernel)
        # gradient niveau 2 
        kernel = np.array([[c,1,0], [1, 0, -1], [0, -1, -c]])/n
        grad2 = self.filtre(img, kernel)
        # gradient niveau 3 
        kernel = np.array([[1,0,-1], [c, 0, -c], [1, 0, -1]])/n
        grad3 = self.filtre(img, kernel)
        # gradient niveau 4 
        kernel = np.array([[-0,-1,-c], [1, 0, -1], [c, 1, 0]])/n
        grad4 = self.filtre(img, kernel)
        # gradient niveau 5 
        kernel = np.array([[-1,-c,-1], [0, 0, 0], [1, c,1]])/n
        grad5 = self.filtre(img, kernel)
        # gradient niveau 6 
        kernel = np.array([[-c,-1,0], [-1, 0, 1], [0, 1, c]])/n
        grad6 = self.filtre(img, kernel)
        # gradient niveau 7
        kernel = np.array([[-1,0,1], [-c, 0, c], [-1, 0, 1]])/n
        grad7 = self.filtre(img, kernel)
        # gradient niveau 8 
        kernel = np.array([[0,1,c], [-1, 0, 1], [-c, -1, 0]])/n
        grad8 = self.filtre(img, kernel)

        # final image
        newimg = np.sqrt(grad1**2 + grad2**2 + grad3**2 + grad4**2 + grad5**2 + grad6**2 + grad7**2 + grad8**2)
        _, newImg = cv2.threshold(newimg, 90, 1,cv2.THRESH_BINARY)
        return newImg
    def roberts(self, img):
        # gradient selon x
        kernel = np.array([[1, 0], 
                         [0, -1]])
        gradx = self.filtre(img, kernel)
        # gradient selon y
        kernel = np.array([[0, 1], 
                         [-1, 0]])
        grady = self.filtre(img, kernel)
        # image final 
        newimg = np.sqrt(gradx**2 + grady**2)
        _ , newImg = cv2.threshold(newimg, 90, 1,cv2.THRESH_BINARY)
        return newImg
    def laplacien(self, img, kernelType):
        # 
        if kernelType == 1:
            kernel = np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]])
        elif kernelType == 2:
            kernel = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])
        elif kernelType == 3:
            kernel = np.array([[1, -2, 1], [-2, 4, -2], [1, -2, 1]])
        else:
            return "the kernel type should be 1,2 or 3"
        img = self.filtre(img, kernel)
        _ , newimg = cv2.threshold(img, 90, 1,cv2.THRESH_BINARY)

        return newimg 

# Morphologie

In [6]:
class Morphologie:
    def __init__(self):
        pass
    def Erosion(self,img, kernel, iteration):
        
        imgBinaire = cv2.threshold(img, 128, 255,cv2.THRESH_BINARY)

        imgBinaire = imgBinaire[1]

        newImg = imgBinaire.copy()


        kw, kh = kernel.shape[0]//2, kernel.shape[1]//2

        k = kernel[kw, kh]

        for k in range(iteration):

            for i in range(kw, imgBinaire.shape[1]-kw):

                for j in range(kh, imgBinaire.shape[0]-kh):

                    if imgBinaire[i,j] == k:

                        if not np.array_equal(kernel, imgBinaire[i-1:i+2,j-1:j+2]):
                            newImg[i,j] = 255


        return newImg
    def Dilatation(img, kernel, iteration):
    
        imgBinaire = cv2.threshold(img, 128, 255,cv2.THRESH_BINARY)

        imgBinaire = imgBinaire[1]

        newImg = imgBinaire.copy()


        kw, kh = kernel.shape[0]//2, kernel.shape[1]//2

        k = kernel[kw, kh]

        for k in range(iteration):

            for i in range(kw, imgBinaire.shape[1]-kw):

                for j in range(kh, imgBinaire.shape[0]-kh):

                    if imgBinaire[i,j] == k:

                        if not imgBinaire[i-1:i+2,j-1:j+2].all():
                            newImg[i,j] = 0
        return newImg
    def Ouverture(img,kernel, intertion):
        # application de l'erosion
        imgErode = Erosion(img,kernel, intertion)
        # application de la dilatation 
        imgDilate = Dilatation(imgErode,kernel, intertion)
        return imgDilate
    def fermeture(img,kernel, iteration):
        # application de la dilatation 
        imgDilate = Dilatation(img,kernel, iteration)
        # application de l'erosion
        imgErode = Erosion(imgDilate,kernel, iteration)

        return imgErode


# Filtrage

In [7]:
class Filtrage:
    def __init__(self):
        pass
    def Lissage(img, kernel_size = 3):
        
        kernel = np.ones((kernel_size,kernel_size))
        
        imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        newimg = np.zeros(imgGray.shape, dtype=np.float32)

        kernel_height, kernel_width = kernel.shape
        kh, kw = kernel_height // 2, kernel_width // 2  

        for i in range(kh, imgGray.shape[0] - kh):
            for j in range(kw, imgGray.shape[1] - kw): 
                sum_value = 0
                for ki in range(kernel_height):
                    for kj in range(kernel_width):
                        sum_value += kernel[ki, kj] * imgGray[i + ki - kh, j + kj - kw]

                newimg[i, j] = sum_value/(kernel.shape[0]**2)

        newimg = np.clip(newimg, 0, 255).astype(np.uint8)

        return newimg
    
    def filtre_gaussien(image, kernel_size=(3, 3), sigma=0):
        return cv2.GaussianBlur(image, kernel_size, sigma)

    def filtre_moyenne(image, kernel_size=(3, 3)):
        return cv2.blur(image, kernel_size)

    def filtre_median(image, kernel_size=3):
        return cv2.medianBlur(image, kernel_size)



# Segmentation

In [8]:
def croissanceReg(image, seed_point, threshold=10):
# Vérifier que l'image est en niveaux de gris
if len(image.shape) != 2:
    raise ValueError("L'image doit être en niveaux de gris.")

# Dimensions de l'image
height, width = image.shape

# Masque pour marquer les pixels visités
segmented_mask = np.zeros_like(image, dtype=np.uint8)

# Liste des pixels à explorer
pixels_to_explore = [seed_point]

# Intensité du pixel de départ
seed_intensity = image[seed_point[1], seed_point[0]]

# Définir les directions des voisins (8-connexité)
neighbors = [(-1, -1), (-1, 0), (-1, 1),
             (0, -1),          (0, 1),
             (1, -1),  (1, 0), (1, 1)]

# Tant qu'il y a des pixels à explorer
while pixels_to_explore:
    # Prendre le premier pixel de la liste
    x, y = pixels_to_explore.pop(0)

    # Vérifier si le pixel est dans les limites de l'image
    if x < 0 or y < 0 or x >= width or y >= height:
        continue

    # Vérifier si le pixel a déjà été visité
    if segmented_mask[y, x] == 1:
        continue

    # Vérifier si l'intensité du pixel est proche de celle du seed
    if abs(int(image[y, x]) - int(seed_intensity)) <= threshold:
        # Marquer le pixel comme faisant partie de la région
        segmented_mask[y, x] = 1

        # Ajouter les voisins à la liste des pixels à explorer
        for dx, dy in neighbors:
            pixels_to_explore.append((x + dx, y + dy))

return segmented_mask

# Create the App

In [ ]:
from PySide6.QtWidgets import QApplication, QWidget, QMainWindow, QFrame, QHBoxLayout, QSizePolicy, QMenu, QFileDialog, QToolBar, QStatusBar, QPushButton, QLabel, QVBoxLayout, QDialog
from PySide6.QtGui import QAction, QIcon, QPixmap, QFont, QColor, QPalette, QImage, QMovie
from PySide6.QtCore import QSize, Qt
import sys
import cv2
import numpy as np
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

class Morphologie:
    def ouverture(self, img, kernel, iterations):
        return cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, iterations=iterations)

    def fermeture(self, img, kernel, iterations):
        return cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, iterations=iterations)

    def erosion(self, img, kernel, iterations):
        return cv2.erode(img, kernel, iterations=iterations)

    def dilatation(self, img, kernel, iterations):
        return cv2.dilate(img, kernel, iterations=iterations)

class HistogramDialog(QDialog):
    def __init__(self, image, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Histogram")
        self.setGeometry(100, 100, 800, 600)

        # Create a matplotlib figure and canvas
        self.figure = Figure()
        self.canvas = FigureCanvas(self.figure)

        # Layout
        layout = QVBoxLayout()
        layout.addWidget(self.canvas)
        self.setLayout(layout)

        # Plot the histogram
        self.plot_histogram(image)

    def plot_histogram(self, image):
        # Calculate histogram using OpenCV
        if len(image.shape) == 2:  # Grayscale image
            hist = cv2.calcHist([image], [0], None, [256], [0, 256])
            self.figure.clear()
            ax = self.figure.add_subplot(111)
            ax.plot(hist, color='black')
            ax.set_title("Grayscale Histogram")
            ax.set_xlabel("Pixel Intensity")
            ax.set_ylabel("Frequency")
        else:  # Color image
            colors = ('b', 'g', 'r')
            self.figure.clear()
            ax = self.figure.add_subplot(111)
            for i, color in enumerate(colors):
                hist = cv2.calcHist([image], [i], None, [256], [0, 256])
                ax.plot(hist, color=color)
            ax.set_title("Color Histogram")
            ax.set_xlabel("Pixel Intensity")
            ax.set_ylabel("Frequency")
        self.canvas.draw()

class MainWindow(QMainWindow):
    def __init__(self, app):
        super().__init__()
        self.morpho = Morphologie()
        self.app = app
        self.current_image = None
        self.setWindowTitle("Image Processing App")
        central_widget = QWidget()
        self.setWindowIcon(QIcon("icone.jpg"))
        central_widget = QWidget()

        # --- Background GIF ---
        self.background_label = QLabel(self)
        self.background_label.setGeometry(0, 0, self.width(), self.height())
        self.movie = QMovie("cat.gif")
        self.background_label.setMovie(self.movie)
        self.movie.start()

        # Menubar and menus
        menu_bar = self.menuBar()
        file_menu = menu_bar.addMenu("&File")
        upload = file_menu.addAction("Upload")
        save = file_menu.addAction("Save")
        delete = file_menu.addAction("Delete")
        exit_app = file_menu.addAction("Exit")

        edge_menu = menu_bar.addMenu("&Extract Edges")
        sobel = edge_menu.addAction("Sobel")
        prewitt = edge_menu.addAction("Prewitt")
        robinson = edge_menu.addAction("Robinson")
        robert = edge_menu.addAction("Robert")
        laplacien = edge_menu.addAction("Laplacien")

        Morpholoogy_menu = menu_bar.addMenu("&Morphology")
        OPENING = Morpholoogy_menu.addAction("Ouverture")
        CLOSING = Morpholoogy_menu.addAction("Fermeture")
        DILATATION = Morpholoogy_menu.addAction("Dilatation")
        EROSION = Morpholoogy_menu.addAction("Erosion")

        filtering_menu = menu_bar.addMenu("&Filtering")
        Gaussien = filtering_menu.addAction("Gaussien")
        Moyenne = filtering_menu.addAction("Moyenne")
        Mediane = filtering_menu.addAction("Mediane")

        seg_menu = menu_bar.addMenu("&Segmentation")
        region_growing_action = seg_menu.addAction("Region Growing")

        histogram_menu = menu_bar.addMenu("&Histogram")
        show_histogram_action = histogram_menu.addAction("Show Histogram")  # Add Histogram action

        help_menu = menu_bar.addMenu("&Help")

        # Connect actions
        upload.triggered.connect(self.upload)
        save.triggered.connect(self.save)
        delete.triggered.connect(self.delete)
        exit_app.triggered.connect(self.ExitApp)

        sobel.triggered.connect(lambda: self.edgeExtractor(1))
        prewitt.triggered.connect(lambda: self.edgeExtractor(2))
        robinson.triggered.connect(lambda: self.edgeExtractor(3))
        robert.triggered.connect(lambda: self.edgeExtractor(4))
        laplacien.triggered.connect(lambda: self.edgeExtractor(5))

        OPENING.triggered.connect(self.ouverture)
        DILATATION.triggered.connect(self.dilatation)
        EROSION.triggered.connect(self.erosion)
        CLOSING.triggered.connect(self.fermeture)

        Gaussien.triggered.connect(lambda: self.apply_filter("gaussien"))
        Moyenne.triggered.connect(lambda: self.apply_filter("moyenne"))
        Mediane.triggered.connect(lambda: self.apply_filter("mediane"))

        region_growing_action.triggered.connect(self.apply_region_growing)
        show_histogram_action.triggered.connect(self.show_histogram)  # Connect Histogram action

        # Status bar
        self.setStatusBar(QStatusBar(self))

        # Labels for images
        self.image_label1 = QLabel("Upload image")
        self.image_label2 = QLabel()

        self.image_label1.setPixmap(QPixmap().scaled(500, 400))
        self.image_label1.setAlignment(Qt.AlignCenter)
        self.image_label2.setPixmap(QPixmap().scaled(500, 400))
        self.image_label2.setAlignment(Qt.AlignCenter)

        self.image_label1.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        self.image_label2.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)

        # Layout
        separator_border = QFrame()
        separator_border.setFrameShape(QFrame.VLine)
        separator_border.setLineWidth(40)
        Main_layout = QHBoxLayout()
        Main_layout.addWidget(self.image_label1)
        Main_layout.addWidget(separator_border)
        Main_layout.addWidget(self.image_label2)

        central_widget.setLayout(Main_layout)
        self.setCentralWidget(central_widget)

        self.set_menu_bar_style(menu_bar)

    def set_menu_bar_style(self, menu_bar):
        menu_bar.setStyleSheet("""
            QMenuBar {
                background-color: #333333;
                color: white;
                font-family: Gotham;
                font: bold 14px;
                border: 0px solid black;
            }
            QMenuBar::item {
                background: transparent;
                padding: 5px 10px;
                border-radius: 5px;
            }
            QMenuBar::item:hover {
                background-color: #555555;
                color: yellow;
            }
            QMenuBar::item:selected {
                background: #FFFFFF;
                color: black;
            }
            QMenuBar::item:pressed {
                background: #FFFFFF;
                color: black;
            }
            QMenu {
                background-color: #333333;
                color: white;
                font: bold 12px;
                border: 0px solid black;
            }
            QMenu::item {
                background: transparent;
                padding: 5px 20px;
                border-radius: 5px;
            }
            QMenu::item:hover {
                background-color: #444444;
            }
            QMenu::item:selected {
                background-color: #FFFFFF;
                color: black;
            }
        """)

    def upload(self):
        defaultPath = self.current_image if self.current_image else ""
        img_path, _ = QFileDialog.getOpenFileName(
            self, "Select Image", defaultPath, "Images (*.png *.jpg *.jpeg *.bmp *.gif)"
        )
        if img_path:
            print(f"Selected image: {img_path}")
            label_width = self.image_label1.width()
            label_height = self.image_label1.height()
            pixmap = QPixmap(img_path).scaled(label_width, label_height, Qt.KeepAspectRatio)
            self.image_label1.setPixmap(pixmap)
            self.current_image = img_path
        return img_path

    def save(self):
        processed_pixmap = self.image_label2.pixmap()
        if processed_pixmap is None:
            self.statusBar().showMessage("No processed image to save!", 5000)
            return

        save_path, _ = QFileDialog.getSaveFileName(
            self, "Save Processed Image", "", "Images (*.png *.jpg *.jpeg)"
        )

        if save_path:
            processed_image = processed_pixmap.toImage()
            if processed_image.save(save_path):
                self.statusBar().showMessage("Image saved successfully!", 5000)
            else:
                self.statusBar().showMessage("Failed to save the image!", 5000)

    def delete(self):
        self.image_label1.clear()
        self.image_label2.clear()
        self.current_image = None
        print("Image deleted and cleared.")

    def ExitApp(self):
        self.app.quit()

    def edgeExtractor(self, m):
        img = cv2.imread(self.current_image, 0)
        if m == 1:
            filtered_image = self.sobel(img)
        elif m == 2:
            filtered_image = self.prewitt(img)
        elif m == 3:
            filtered_image = self.robinson(img, 3)
        elif m == 4:
            filtered_image = self.roberts(img)
        else:
            filtered_image = self.laplacien(img, 1)

        qimg = QImage(filtered_image.data, filtered_image.shape[1], filtered_image.shape[0], QImage.Format_Grayscale8)
        pixmap = QPixmap.fromImage(qimg)
        pixmap = QPixmap(pixmap).scaled(500, 400)
        self.image_label2.setPixmap(pixmap)

    def ouverture(self):
        if self.current_image:
            img = cv2.imread(self.current_image, 0)
            kernel = np.ones((5, 5), np.uint8)
            iteration = 1
            result = self.morpho.ouverture(img, kernel, iteration)
            self.display_image(result)

    def fermeture(self):
        if self.current_image:
            img = cv2.imread(self.current_image, 0)
            kernel = np.ones((5, 5), np.uint8)
            iteration = 1
            result = self.morpho.fermeture(img, kernel, iteration)
            self.display_image(result)

    def erosion(self):
        if self.current_image:
            img = cv2.imread(self.current_image, 0)
            kernel = np.ones((5, 5), np.uint8)
            iteration = 1
            result = self.morpho.erosion(img, kernel, iteration)
            self.display_image(result)

    def dilatation(self):
        if self.current_image:
            img = cv2.imread(self.current_image, 0)
            kernel = np.ones((5, 5), np.uint8)
            iteration = 1
            result = self.morpho.dilatation(img, kernel, iteration)
            self.display_image(result)

    def display_image(self, img):
        qimg = QImage(img.data, img.shape[1], img.shape[0], QImage.Format_Grayscale8)
        pixmap = QPixmap.fromImage(qimg)
        label_width = self.image_label2.width()
        label_height = self.image_label2.height()
        pixmap = pixmap.scaled(label_width, label_height, Qt.KeepAspectRatio)
        self.image_label2.setPixmap(pixmap)

    def resizeEvent(self, event):
        self.background_label.setGeometry(0, 0, self.width(), self.height())
        self.movie.setScaledSize(QSize(self.width(), self.height()))
        super().resizeEvent(event)

    def apply_filter(self, filter_type):
        if self.current_image:
            img = cv2.imread(self.current_image, 0)
            if filter_type == "gaussien":
                result = cv2.GaussianBlur(img, (5, 5), 0)
            elif filter_type == "moyenne":
                result = cv2.blur(img, (5, 5))
            elif filter_type == "mediane":
                result = cv2.medianBlur(img, 5)
            self.display_image(result)

    def apply_region_growing(self):
        if self.current_image:
            img = cv2.imread(self.current_image, 0)
            seed_point = (100, 100)  # Example seed point (x, y)
            threshold = 10  # Example threshold
            segmented_mask = self.croissanceReg(img, seed_point, threshold)
            self.display_image(segmented_mask * 255)  # Convert mask to binary image

    def croissanceReg(self, image, seed_point, threshold=10):
        if len(image.shape) != 2:
            raise ValueError("L'image doit être en niveaux de gris.")

        height, width = image.shape
        segmented_mask = np.zeros_like(image, dtype=np.uint8)
        pixels_to_explore = [seed_point]
        seed_intensity = image[seed_point[1], seed_point[0]]

        neighbors = [(-1, -1), (-1, 0), (-1, 1),
                     (0, -1),          (0, 1),
                     (1, -1),  (1, 0), (1, 1)]

        while pixels_to_explore:
            x, y = pixels_to_explore.pop(0)
            if x < 0 or y < 0 or x >= width or y >= height:
                continue
            if segmented_mask[y, x] == 1:
                continue
            if abs(int(image[y, x]) - int(seed_intensity)) <= threshold:
                segmented_mask[y, x] = 1
                for dx, dy in neighbors:
                    pixels_to_explore.append((x + dx, y + dy))

        return segmented_mask

    def show_histogram(self):
        if self.current_image:
            img = cv2.imread(self.current_image)
            histogram_dialog = HistogramDialog(img, self)
            histogram_dialog.exec()

# Main application
app = QApplication.instance()
if not app:
    app = QApplication([])
window = MainWindow(app)
window.showMaximized()
sys.exit(app.exec())

Selected image: C:/Users/dell/Downloads/5.jpg
Image deleted and cleared.
Selected image: C:/Users/dell/Downloads/5.jpg
Image deleted and cleared.
Selected image: C:/Users/dell/Downloads/5.jpg
Image deleted and cleared.
Selected image: C:/Users/dell/Downloads/Sans titre.png


# Test componants 

In [6]:
from PySide6.QtWidgets import QApplication, QWidget
import sys

app = QApplication(sys.argv)
window = QWidget()
window.setWindowTitle("My First PySide6 App")  # Optional: Add a title to the window
window.resize(400, 300)  # Optional: Set initial size
window.show()
sys.exit(app.exec())


SystemExit: 0

C:\Users\Hassan Kerroumi\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
from PySide6.QtWidgets import QApplication, QWidget, QMainWindow, QPushButton
import sys

"""app = QApplication(sys.argv)
window = QMainWindow()
window.setWindowTitle("My First PySide6 App")  # Optional: Add a title to the window
window.resize(400, 300) 
button = QPushButton()"""
class ButtonHolder(QMainWindow):
    
    def __init__ (self):
        super().__init__()
        self.setWindowTitle("Button holder App ")
        button = QPushButton("Press Me")
        self.setCentralWidget(button)
        self.resize(400, 300)
app = QApplication.instance()  
if not app:  
    app = QApplication(sys.argv)        
window = ButtonHolder()
window.show()
sys.exit(app.exec())


SystemExit: 0

In [9]:
from PySide6.QtWidgets import QApplication, QWidget, QMainWindow, QPushButton


def clicked_button():
    print("You clicked the button didn't you ")
app = QApplication.instance()  
if not app:  
    app = QApplication(sys.argv) 
button = QPushButton("Press Me ")

button.clicked.connect(clicked_button)
button.resize(200,200)
button.show()
sys.exit(app.exec())


You clicked the button didn't you 
You clicked the button didn't you 
You clicked the button didn't you 
You clicked the button didn't you 
You clicked the button didn't you 
You clicked the button didn't you 
You clicked the button didn't you 
You clicked the button didn't you 
You clicked the button didn't you 
You clicked the button didn't you 


SystemExit: 0

In [10]:
from PySide6.QtWidgets import QApplication, QWidget, QMainWindow, QPushButton


def clicked_button(data):
    print("You clicked the button didn't you ", data)
app = QApplication.instance()  
if not app:  
    app = QApplication(sys.argv) 
button = QPushButton("Press Me ")
button.setCheckable(True)
button.clicked.connect(clicked_button)
button.resize(200,200)
button.show()
sys.exit(app.exec())


You clicked the button didn't you  True


SystemExit: 0

In [14]:
from PySide6.QtWidgets import QApplication,QSlider 
from PySide6.QtCore import Qt


def respond_to_slider(data):
    print("slider move to ", data)
app = QApplication.instance()  
if not app:  
    app = QApplication(sys.argv) 
slider = QSlider(Qt.Horizontal)
slider.setMinimum(0)
slider.setMaximum(100)
slider.setValue(25)
slider.valueChanged.connect(respond_to_slider)
slider.show()
sys.exit(app.exec())


SystemExit: 0

In [22]:
from PySide6.QtWidgets import QApplication, QWidget, QMainWindow, QHBoxLayout

class RocWidget(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("RocWidget")
        
        button1 = QPushButton("Buttton1")
        button1.clicked.connect(self.b1_clicked)
        button2 = QPushButton("Buttton2")
        button2.clicked.connect(self.b2_clicked)
        button_layout = QHBoxLayout()
        button_layout.addWidget(button1)
        button_layout.addWidget(button2)
        self.setLayout(button_layout)
    def b1_clicked(self):
        print("button 1 clicked")
    def b2_clicked(self):
        print("button 2 clicked")   
app = QApplication.instance()  
if not app:  
    app = QApplication(sys.argv) 
button = RocWidget()

button.show()
sys.exit(app.exec())      
    

button 1 clicked
button 2 clicked


SystemExit: 0

# Create the main window 

## Widget 

In [51]:
from PySide6.QtWidgets import QApplication, QWidget, QLabel, QHBoxLayout, QVBoxLayout, QPushButton 
from PySide6.QtGui import QPixmap

class Widget(QWidget):
    def __init__(self,):
        super().__init__()
        self.setWindowTitle("QLabel Image Demo")
        
        image_label = QLabel()
        image_label.setPixmap(QPixmap("opencv.png"))
        image_label1 = QLabel()
        image_label1.setPixmap(QPixmap("birds.jpg"))
        layout = QHBoxLayout()
        layout.addWidget(image_label)
        layout.addWidget(image_label1)
        
        self.setLayout(layout)
        

In [52]:
from PySide6.QtWidgets import QApplication
import sys 

app = QApplication.instance()  
if not app:  
    app = QApplication(sys.argv) 
window = Widget()

window.show()
sys.exit(app.exec())

SystemExit: 0

# List Widget 

In [89]:
from PySide6.QtWidgets import QApplication, QWidget, QLabel, QHBoxLayout, QVBoxLayout, QListWidget, QAbstractItemView 
from PySide6.QtGui import QPixmap

class Widget(QWidget):
    def __init__(self,):
        super().__init__()
        self.setWindowTitle("list widget Demo")
        self.list_widget = QListWidget(self)
        self.list_widget.setSelectionMode(QAbstractItemView.MultiSelection)
        self.list_widget.addItem("One")
        self.list_widget.addItems(["Two", "Three", "Four"])
        
        self.list_widget.currentItemChanged.connect(self.current_item_changed)
        self.list_widget.currentTextChanged.connect(self.current_text_changed)
        v_layout = QVBoxLayout()
        v_layout.addWidget(self.list_widget)
        
        self.setLayout(v_layout)
    def current_item_changed(self, item):
        print("current item : ", item.text())
    def current_text_changed(self, text):
        print("current text changed : ",text)

In [90]:
from PySide6.QtWidgets import QApplication
import sys 

app = QApplication.instance()  
if not app:  
    app = QApplication(sys.argv) 
window = Widget()

window.show()
sys.exit(app.exec())

current item :  One
current text changed :  One
current item :  Three
current text changed :  Three
current item :  Four
current text changed :  Four
current item :  Two
current text changed :  Two
current item :  One
current text changed :  One


SystemExit: 0

In [14]:
!pip install nbimporter

In [20]:
import Computer_Vision 
hello()

ModuleNotFoundError: No module named 'Computer_Vision'